In [48]:
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
import rasterio
import xml.etree.ElementTree as ET
import pds4_tools

from scipy import signal
from scipy.ndimage import gaussian_filter

In [57]:
structures = pds4_tools.read('data2017/venus_2017_north_ocp.xml')
structures.info()
data = structures[0].data
dtype = data.dtype
lines = data.shape[0]
samples = data.shape[1]

Processing label: data2017/venus_2017_north_ocp.xml
Now processing a Array_2D_Map structure: image_object
0   Array_2D_Map  'image_object'  2 axes, 12000 x 20000


In [6]:
# ## parse the XML label files
# tree = ET.parse('data2020/venus_2020_north_ocp.xml')
# root = tree.getroot()

# ## doesn't work. fml bruce. thanks for using same tags for multiple things
# label_dict = {}


# for element in root.iter():
#     label_dict[element.tag.split('}')[1]] = element.text
# #         print(element.tag.split('}')[1] , element.attrib,element.text)
#     if element.attrib != {}:
# #             print(element.items())
#         for item in element.items():
#             label_dict[element.tag.split('}')[1]+"_"+item[0]] = item[1]
# #                 print(element.tag.split('}')[1]+"_"+item[0])
# #                 print(element.tag.split('}')[1]+"_"+item[1])
# #                 print(item)
# #             attrib_keys = element.keys()
# #             attrib_values = element.values()
# #             if len(attrib_keys) == 1:
# #                 print(element.tag.split('}')[1]+"_"+attrib_keys[0])

In [20]:
## Reading Arecibo images without PDS4
# lines = 12000
# samples = 20000
# filename = 'data2020/venus_2020_north_inc.img'
# datatype = np.float32
# f = open(filename, 'rb')
# data_raw = np.fromfile(f, datatype)
# data = data_raw.reshape(lines, samples)

In [ ]:
## transforming Arecibo data
WKT_string = PROJCS["Venus_Sinusoidal",
    GEOGCS["Venus 2000",
        DATUM["D_Venus_2000",
            SPHEROID["Venus_2000_IAU_IAG",6051800.0,0.0]],
        PRIMEM["Greenwich",0],
        UNIT["Decimal_Degree",0.0174532925199433]],
    PROJECTION["Equidistant_Cylindrical"],
    PARAMETER["False_Easting",0],
    PARAMETER["False_Northing",0],
    PARAMETER["Central_Meridian",335.0],
    PARAMETER["Standard_Parallel_1",0],
    UNIT["Meter",1]]
transform = rasterio.transform.from_bounds(254.256, south, east, north, width, height)
transform = rasterio.transform.from_bounds(254.256, -25.0, 37.213, 80.845, samples, lines)

In [58]:
outfile = "data2017/north_ocp.tif"
outds = rasterio.open(outfile, 'w', driver='GTiff', 
                  height = lines, 
                  width = samples, 
                  count=1, 
                  crs = None, 
                  dtype = dtype,
                  transform = None,
                  compress='lzw',
                  nodata = 9999)
outds.write(data, 1)
outds.close()

In [59]:
avgdata = np.zeros_like(data)
gaussian_filter(data, sigma=1, output=avgdata)
outfile_gau = "data2017/north_ocp_sigma1.tif"
outds_gau = rasterio.open(outfile_gau, 'w', driver='GTiff', 
                  height = lines, 
                  width = samples, 
                  count=1, 
                  crs = None, 
                  dtype = dtype,
                  transform = None,
                  compress='lzw',
                  nodata = 9999)
outds_gau.write(avgdata, 1)
outds_gau.close()